# Deep Q-Network (DQN)
---
In this notebook, you will implement a DQN agent with OpenAI Gym's LunarLander-v2 environment.

### 1. Import the Necessary Packages

In [17]:
import random
import torch
import numpy as np
from infra.game import Game
from infra.util import getValidMoves
import csv
import math
from collections import deque
import matplotlib.pyplot as plt
from dqn_agent import Agent
from tqdm import tqdm_notebook
%matplotlib inline

### 2. Instantiate the Environment and Agent

Initialize the environment in the code cell below.

In [23]:
from dqn_agent import Agent
# one suit case 
# state space: initial state 104!/(8!)^13 * (C_54^10 + ... + C_104^10) +
# 91!/(8!)^12 * (C_41^10 + ... + C_91^10)... + 13!*C_13^10 + 1
# action space: deal and move 10 columns move to the other 9 cols, move at most 12 cards, 10*9*12+1
# According to this method, we can simply say the size of state space equals to 104 
# And one way is we record the know of the numbr of the cards (flip equal to 0) and record 
# 10 columns (num)  + stock number + how many completed
# the action should be move or deal 
# (avoid too complex action space, we set every move we only move the valid move 0)

agent = Agent(state_size=115, action_size=1081, seed=0)
# watch an untrained agent
game = Game(0)
game.createGame((0, 0, 0, 0))
game.startGame()
print(game)
for i in range(3):
    moves = getValidMoves(game.getVisibleState())
    state = np.array(game.getStateForDQN())
    # how to use agent to get the actions / how to modify the actions
    action = agent.act(state, moves)
    game.performMovesForDQN(action)
    print(game)


-♤  J- -♤  7- -♤  2- -♤  6- -♤  8- -♤  A- -♤  J- -♤  K- -♤  K- -♤ 10-
-♤  7- -♤ 10- -♤  2- -♤  K- -♤  5- -♤  J- -♤  Q- -♤  K- -♤  2- -♤  7-
-♤  4- -♤  K- -♤  6- -♤  A- -♤  9- -♤ 10- -♤  4- -♤  9- -♤  7- -♤  8-
-♤  3- -♤  4- -♤  2- -♤  A- -♤  4- -♤  5- -♤  2- -♤  9- -♤  8- -♤  2-
-♤  Q- -♤  8- -♤  J- -♤  K-  ♤  J   ♤  A   ♤  3   ♤  6   ♤  9   ♤  3 
 ♤  9   ♤  A   ♤  Q   ♤  5                                           
Game#0 Score: 500   Moves: 0   Completed: 0   Undealt: 5
-♤  J- -♤  7- -♤  2- -♤  6- -♤  8- -♤  A- -♤  J- -♤  K- -♤  K- -♤ 10-
-♤  7- -♤ 10- -♤  2- -♤  K- -♤  5- -♤  J- -♤  Q- -♤  K- -♤  2- -♤  7-
-♤  4- -♤  K- -♤  6- -♤  A- -♤  9- -♤ 10- -♤  4- -♤  9- -♤  7- -♤  8-
-♤  3- -♤  4- -♤  2- -♤  A- -♤  4- -♤  5- -♤  2- -♤  9- -♤  8- -♤  2-
-♤  Q- -♤  8- -♤  J-  ♤  K   ♤  J   ♤  A   ♤  3   ♤  6   ♤  9   ♤  3 
 ♤  9   ♤  A   ♤  Q                               ♤  5               
Game#0 Score: 499   Moves: 1   Completed: 0   Undealt: 5
-♤  J- -♤  7- -♤  2- -♤  6- -♤  8- -♤  A- -♤  

### 3. Train the Agent with DQN

Run the code cell below to train the agent from scratch.  You are welcome to amend the supplied values of the parameters in the function, to try to see if you can get better performance!

Alternatively, you can skip to the next step below (**4. Watch a Smart Agent!**), to load the saved model weights from a pre-trained agent.

In [ ]:
def dqn(n_episodes=2000, eps_start=0.5, eps_end=0.1, eps_decay=0.99):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    numCompleted = 0
    eps = eps_start                    # initialize epsilon
    for i in tqdm_notebook(range(1000)):
        game = Game(i)
        game.createGame((0, 0, 0, 0))
        game.startGame()
        score = game.score
        moves = game.getValidMoves()
        while not game.won() and moves:
            state = np.array(game.getStateForDQN())
            game.performMoves(moves[0])
            reward = game.score - score
            next_state = np.array(game.getStateForDQN())
            if moves[0][1]=='deal':
                action = 1080
            else:
                _, _, ncards, from_stack, to_stack = moves[0]
                f = from_stack.id-1
                t = to_stack.id-1
                action = (f*9+t-(f<t))*12+ncard-1
            #print(state, action, reward, next_state, game.won())
            agent.step(state, action, reward, next_state, game.won())
            moves = game.getValidMoves()
            score = game.score
        #print(game.score, game.won())
    for i_episode in tqdm_notebook(range(1, n_episodes+1)):
        game = Game(i_episode)
        game.createGame((0, 0, 0, 0))
        game.startGame()
        score = game.score
        moves = getValidMoves(game.getVisibleState())
        # one game till win the game or cannot move
        while not game.won() and moves:
            state = np.array(game.getStateForDQN())
            action = agent.act(state, moves, eps)
            game.performMovesForDQN(action)
            next_state = np.array(game.getStateForDQN())
            reward = game.score - score
            won = game.won()
            if won:
#                 reward += 10000
                numCompleted += 1
            agent.step(state, action, reward, next_state, game.won())
            moves = getValidMoves(game.getVisibleState())
            score = game.score
        scores.append(score)              # save most recent score
        scores_window.append(score)       # save most recent score
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
#         print('\rEpisode {}\tAverage Score: {:.2f} \tCompleted: {}'.format(i_episode, score, completed), end="")
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f} \tCompleted: {}, eps: {}'.format(i_episode, np.mean(scores_window), numCompleted/i_episode, eps))
        if numCompleted * 1.0 / i_episode > 0.45:
            print('\nEnvironment solved in {:d} episodes!\tNumber of complted game: {:.2f}'.format(i_episode, numCompleted))
            torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')
            break
        elif i_episode == n_episodes:
            print('\nRunning all {:d} episodes!\tNumber of complted game: {:.2f}'.format(i_episode, numCompleted))
            torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')
    return scores, numCompleted

agent = Agent(state_size=115, action_size=1081, seed=0)
scores, numCompleted = dqn()

# plot the scores
print(numCompleted)
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

### 4. Watch a Smart Agent!

In the next code cell, you will load the trained weights from file to watch a smart agent!

In [16]:
# load the weights from file
agent.qnetwork_local.load_state_dict(torch.load('checkpoint.pth'))

game = Game(10000)
game.createGame((0, 0, 0, 0))
game.startGame()
print(game)
moves = getValidMoves(game.getVisibleState())
while not game.won() and moves:
    state = np.array(game.getStateForDQN())
    action = agent.act(state, moves)
    game.performMovesForDQN(action)
    moves = getValidMoves(game.getVisibleState())
print(game)
print('Game#%d [%s] Score: %d   Moves: %d   Completed: %d' %
                      (game.id, 'Won' if game.won() else 'Lost', game.score, game.moves, game.completed))

# baseline
game = Game(10000)
game.createGame((0, 0, 0, 0))
game.startGame()
moves = game.getValidMoves()
while not game.won() and moves:
    game.performMoves(moves[0])
    moves = game.getValidMoves()
print(game)
print('Game#%d [%s] Score: %d   Moves: %d   Completed: %d' %
                      (game.id, 'Won' if game.won() else 'Lost', game.score, game.moves, game.completed))

-♤ 10- -♤  K- -♤  A- -♤  3- -♤  8- -♤  7- -♤  4- -♤ 10- -♤  9- -♤  6-
-♤  2- -♤  9- -♤  Q- -♤  5- -♤  3- -♤ 10- -♤  6- -♤  Q- -♤  5- -♤  5-
-♤  9- -♤  A- -♤  3- -♤  9- -♤  7- -♤  K- -♤  Q- -♤  3- -♤  2- -♤  K-
-♤  9- -♤  4- -♤ 10- -♤  8- -♤  A- -♤  8- -♤  7- -♤  7- -♤ 10- -♤  2-
-♤  7- -♤  3- -♤  K- -♤  Q-  ♤  8   ♤  3   ♤  9   ♤  7   ♤  Q   ♤  2 
 ♤ 10   ♤  A   ♤  Q   ♤  2                                           
Game#10000 Score: 500   Moves: 0   Completed: 0   Undealt: 5
-♤ 10- -♤  K- -♤  A- -♤  3- -♤  8- -♤  7- -♤  4- -♤ 10- -♤  9- -♤  6-
-♤  2- -♤  9- -♤  Q- -♤  5- -♤  3- -♤ 10- -♤  6- -♤  Q- -♤  5- -♤  5-
-♤  9- -♤  A- -♤  3- -♤  9- -♤  7- -♤  K- -♤  Q- -♤  3- -♤  2- -♤  K-
-♤  9- -♤  4- -♤ 10- -♤  8- -♤  A- -♤  8- -♤  7- -♤  7- -♤ 10- -♤  2-
-♤  7- -♤  3- -♤  K- -♤  Q-  ♤  8   ♤  3   ♤  9   ♤  7   ♤  Q   ♤  2 
 ♤ 10   ♤  A   ♤  Q   ♤  2   ♤  8   ♤  2   ♤  J   ♤  6   ♤  5   ♤  5 
 ♤ 10   ♤  6   ♤  J   ♤  J   ♤  4   ♤  K   ♤  J   ♤  5   ♤ 10   ♤  A 
 ♤  4   ♤  J          ♤  J   

In [13]:
high_score = 0
win_count = 0
count = 1000
agent.qnetwork_local.load_state_dict(torch.load('checkpoint.pth'))

In [14]:
try:
    with open('output/dqn-1suit.csv', 'w') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(('ID', 'Result', 'Score', '#Moves', '#Stacks'))
        for i in range(count):
            game = Game(i)
            game.createGame((0, 0, 0, 0))
            game.startGame()
            print(game)
            moves = getValidMoves(game.getVisibleState())
            while not game.won() and moves:
                state = np.array(game.getStateForDQN())
                action = agent.act(state, moves)
                game.performMovesForDQN(action)
                if game.won():
                    high_score = max(high_score, game.score)
                    win_count += 1
                moves = getValidMoves(game.getVisibleState())
            print('Game#%d [%s] Score: %d   Moves: %d   Completed: %d' %
                  (game.id, 'Won' if game.won() else 'Lost', game.score, game.moves, game.completed))
            writer.writerow(map(str, (game.id, 'Won' if game.won()
                                      else 'Lost', game.score, game.moves, game.completed)))
except KeyboardInterrupt:
    pass
print('Won %d/%d games, high score is %d' % (win_count, count, high_score))


-♤  K- -♤  9- -♤  A- -♤  Q- -♤  9- -♤  K- -♤  3- -♤  7- -♤  8- -♤  A-
-♤  3- -♤  Q- -♤  7- -♤ 10- -♤  2- -♤  3- -♤  2- -♤  J- -♤  8- -♤  A-
-♤  A- -♤  5- -♤  4- -♤  5- -♤  8- -♤  K- -♤  3- -♤  K- -♤  J- -♤  6-
-♤ 10- -♤  2- -♤  4- -♤  A- -♤  J- -♤ 10- -♤  3- -♤  Q- -♤  9- -♤  4-
-♤  3- -♤  8- -♤  6- -♤  A-  ♤  9   ♤  6   ♤  K   ♤  9   ♤  3   ♤  A 
 ♤  5   ♤  5   ♤ 10   ♤  6                                           
Game#0 Score: 500   Moves: 0   Completed: 0   Undealt: 5
Game#0 [Lost] Score: 458   Moves: 42   Completed: 0
-♤ 10- -♤  Q- -♤  A- -♤  3- -♤  3- -♤ 10- -♤  2- -♤  8- -♤  J- -♤  2-
-♤  A- -♤  3- -♤  6- -♤  3- -♤  K- -♤  A- -♤  Q- -♤ 10- -♤  4- -♤  Q-
-♤  K- -♤  J- -♤  6- -♤  J- -♤  6- -♤  5- -♤  7- -♤  Q- -♤  5- -♤  4-
-♤  2- -♤  8- -♤  A- -♤  A- -♤  6- -♤  Q- -♤  7- -♤  4- -♤  K- -♤  9-
-♤ 10- -♤  J- -♤  7- -♤  6-  ♤  2   ♤  Q   ♤  6   ♤  K   ♤  9   ♤  8 
 ♤  8   ♤  5   ♤  9   ♤  J                                           
Game#1 Score: 500   Moves: 0   Completed: 0   Undea

Game#19 [Lost] Score: 482   Moves: 18   Completed: 0
-♤  A- -♤ 10- -♤  4- -♤  9- -♤  7- -♤  4- -♤  5- -♤  3- -♤  A- -♤  8-
-♤  J- -♤  7- -♤  9- -♤  8- -♤  A- -♤  5- -♤  9- -♤  K- -♤ 10- -♤  2-
-♤  2- -♤  8- -♤  6- -♤  A- -♤  6- -♤  K- -♤  5- -♤  J- -♤  J- -♤  Q-
-♤ 10- -♤ 10- -♤  4- -♤  2- -♤  3- -♤  8- -♤  Q- -♤  J- -♤  7- -♤  K-
-♤  A- -♤  5- -♤  K- -♤  6-  ♤ 10   ♤  Q   ♤ 10   ♤  4   ♤  8   ♤  K 
 ♤  7   ♤  7   ♤  9   ♤  Q                                           
Game#20 Score: 500   Moves: 0   Completed: 0   Undealt: 5
Game#20 [Lost] Score: 491   Moves: 9   Completed: 0
-♤  9- -♤  8- -♤  J- -♤  A- -♤  Q- -♤  6- -♤ 10- -♤  4- -♤  Q- -♤  3-
-♤  5- -♤  K- -♤  7- -♤  2- -♤  3- -♤  7- -♤  6- -♤  6- -♤  K- -♤  K-
-♤  J- -♤  3- -♤ 10- -♤  A- -♤  2- -♤  J- -♤  Q- -♤  A- -♤ 10- -♤  9-
-♤  Q- -♤  J- -♤  Q- -♤  4- -♤  7- -♤  3- -♤  J- -♤ 10- -♤  A- -♤  3-
-♤  7- -♤  K- -♤  6- -♤  4-  ♤  2   ♤  3   ♤  2   ♤  K   ♤  7   ♤  8 
 ♤  K   ♤  8   ♤  2   ♤  4                                         

Game#37 [Lost] Score: 549   Moves: 52   Completed: 1
-♤  6- -♤  J- -♤ 10- -♤  Q- -♤  7- -♤  7- -♤  8- -♤  6- -♤  Q- -♤  9-
-♤  9- -♤ 10- -♤  8- -♤  Q- -♤ 10- -♤  A- -♤  Q- -♤  J- -♤  A- -♤  7-
-♤  5- -♤  4- -♤  2- -♤  A- -♤  K- -♤  5- -♤  K- -♤  8- -♤ 10- -♤  3-
-♤  2- -♤  6- -♤  K- -♤  8- -♤  9- -♤  J- -♤  3- -♤  J- -♤  4- -♤  6-
-♤  8- -♤  A- -♤  4- -♤  3-  ♤  6   ♤  9   ♤  2   ♤  5   ♤  5   ♤  2 
 ♤  8   ♤  3   ♤  9   ♤  J                                           
Game#38 Score: 500   Moves: 0   Completed: 0   Undealt: 5
Game#38 [Lost] Score: 472   Moves: 28   Completed: 0
-♤  4- -♤  A- -♤  K- -♤  6- -♤  2- -♤ 10- -♤  7- -♤  4- -♤  2- -♤  7-
-♤  9- -♤  9- -♤ 10- -♤  9- -♤  8- -♤  5- -♤  5- -♤  6- -♤  A- -♤  7-
-♤  J- -♤ 10- -♤  4- -♤  2- -♤ 10- -♤  4- -♤  7- -♤  Q- -♤  A- -♤  9-
-♤  5- -♤  3- -♤  5- -♤  6- -♤  K- -♤  8- -♤  2- -♤  8- -♤  J- -♤  3-
-♤  9- -♤  J- -♤  K- -♤  5-  ♤  4   ♤  3   ♤  8   ♤  7   ♤  4   ♤  5 
 ♤  4   ♤  J   ♤  3   ♤  4                                        

Game#54 [Lost] Score: 468   Moves: 32   Completed: 0
-♤  5- -♤  5- -♤  4- -♤ 10- -♤  4- -♤  5- -♤  6- -♤ 10- -♤  3- -♤  8-
-♤ 10- -♤  4- -♤  J- -♤  J- -♤  5- -♤  Q- -♤  Q- -♤  8- -♤  Q- -♤  8-
-♤  Q- -♤  K- -♤  K- -♤  4- -♤  4- -♤  2- -♤  7- -♤  6- -♤  3- -♤  7-
-♤  3- -♤ 10- -♤  2- -♤  7- -♤  K- -♤  6- -♤  K- -♤  9- -♤ 10- -♤  3-
-♤  K- -♤  8- -♤  8- -♤  A-  ♤  9   ♤ 10   ♤  5   ♤  8   ♤  7   ♤  J 
 ♤  A   ♤  8   ♤  2   ♤  A                                           
Game#55 Score: 500   Moves: 0   Completed: 0   Undealt: 5
Game#55 [Lost] Score: 493   Moves: 7   Completed: 0
-♤  8- -♤  8- -♤  2- -♤  6- -♤  3- -♤  4- -♤  4- -♤  A- -♤  Q- -♤  A-
-♤  9- -♤  3- -♤  5- -♤  2- -♤  3- -♤  2- -♤  K- -♤ 10- -♤ 10- -♤  K-
-♤  9- -♤  Q- -♤  2- -♤  9- -♤  K- -♤  5- -♤  K- -♤  J- -♤  A- -♤  4-
-♤  J- -♤  J- -♤  9- -♤  A- -♤  J- -♤  3- -♤  Q- -♤ 10- -♤  Q- -♤  2-
-♤ 10- -♤  8- -♤  7- -♤  9-  ♤  3   ♤ 10   ♤  7   ♤  5   ♤  6   ♤  Q 
 ♤  7   ♤  Q   ♤  6   ♤  7                                         

Game#70 [Lost] Score: 486   Moves: 14   Completed: 0
-♤ 10- -♤  8- -♤  8- -♤  J- -♤  5- -♤  K- -♤  3- -♤  A- -♤  Q- -♤  9-
-♤  Q- -♤  7- -♤  4- -♤  A- -♤  A- -♤  K- -♤  3- -♤  4- -♤  Q- -♤  6-
-♤  Q- -♤  K- -♤  2- -♤ 10- -♤  8- -♤  4- -♤  J- -♤  7- -♤  A- -♤  6-
-♤  6- -♤  Q- -♤ 10- -♤  5- -♤  3- -♤  2- -♤  5- -♤  8- -♤  7- -♤  6-
-♤ 10- -♤  3- -♤  7- -♤  J-  ♤  4   ♤  5   ♤  A   ♤  3   ♤  6   ♤  K 
 ♤  2   ♤  2   ♤  Q   ♤  8                                           
Game#71 Score: 500   Moves: 0   Completed: 0   Undealt: 5
Game#71 [Lost] Score: 476   Moves: 24   Completed: 0
-♤  4- -♤  K- -♤  2- -♤  3- -♤  8- -♤  3- -♤  4- -♤  6- -♤  K- -♤  K-
-♤ 10- -♤  7- -♤  K- -♤  7- -♤ 10- -♤  2- -♤  8- -♤  9- -♤  8- -♤  J-
-♤  Q- -♤  Q- -♤  6- -♤  4- -♤  8- -♤  3- -♤  9- -♤ 10- -♤  9- -♤ 10-
-♤  K- -♤  3- -♤  8- -♤ 10- -♤  J- -♤  2- -♤  5- -♤  6- -♤  9- -♤  6-
-♤  6- -♤  4- -♤  3- -♤  2-  ♤  6   ♤  7   ♤  8   ♤  K   ♤ 10   ♤  A 
 ♤  3   ♤  9   ♤  5   ♤  9                                        

...

Game#921 [Lost] Score: 571   Moves: 30   Completed: 1
-♤  K- -♤  2- -♤ 10- -♤ 10- -♤  4- -♤  4- -♤  4- -♤  Q- -♤  6- -♤ 10-
-♤  2- -♤  6- -♤  K- -♤  9- -♤  K- -♤  Q- -♤  8- -♤  6- -♤  7- -♤  J-
-♤  Q- -♤ 10- -♤  Q- -♤  3- -♤ 10- -♤  8- -♤  5- -♤  2- -♤  4- -♤  A-
-♤  2- -♤  J- -♤  5- -♤  7- -♤  4- -♤  3- -♤  5- -♤  A- -♤  Q- -♤  2-
-♤  7- -♤  K- -♤  6- -♤  6-  ♤ 10   ♤  5   ♤  3   ♤  9   ♤  6   ♤  J 
 ♤  3   ♤  3   ♤  9   ♤  3                                           
Game#922 Score: 500   Moves: 0   Completed: 0   Undealt: 5
Game#922 [Lost] Score: 477   Moves: 23   Completed: 0
-♤  6- -♤  6- -♤  3- -♤  Q- -♤  3- -♤  2- -♤  A- -♤  9- -♤  8- -♤  A-
-♤  4- -♤  7- -♤ 10- -♤  A- -♤  J- -♤  Q- -♤  K- -♤  3- -♤  5- -♤  K-
-♤ 10- -♤  9- -♤  K- -♤  J- -♤  J- -♤  8- -♤  6- -♤ 10- -♤  3- -♤  2-
-♤  9- -♤  8- -♤  4- -♤  5- -♤  7- -♤  5- -♤  Q- -♤  7- -♤  2- -♤  2-
-♤  8- -♤  A- -♤  3- -♤  8-  ♤  A   ♤ 10   ♤  J   ♤  6   ♤  7   ♤  7 
 ♤  5   ♤  9   ♤  K   ♤  4                                     

Game#938 [Lost] Score: 475   Moves: 25   Completed: 0
-♤  8- -♤ 10- -♤  K- -♤  2- -♤  4- -♤  A- -♤  Q- -♤  4- -♤  9- -♤  5-
-♤  6- -♤  Q- -♤  6- -♤  2- -♤  9- -♤  3- -♤  Q- -♤  5- -♤  5- -♤  7-
-♤  4- -♤  6- -♤  A- -♤  K- -♤  2- -♤ 10- -♤  Q- -♤  3- -♤  5- -♤  K-
-♤  7- -♤  9- -♤  2- -♤  3- -♤ 10- -♤ 10- -♤  K- -♤  9- -♤  2- -♤  9-
-♤  6- -♤  6- -♤  7- -♤  6-  ♤  3   ♤  8   ♤  7   ♤  7   ♤  8   ♤  J 
 ♤  8   ♤  K   ♤  8   ♤  3                                           
Game#939 Score: 500   Moves: 0   Completed: 0   Undealt: 5
Game#939 [Lost] Score: 573   Moves: 28   Completed: 1
-♤  4- -♤  A- -♤  3- -♤ 10- -♤  4- -♤  J- -♤  4- -♤  4- -♤  2- -♤  7-
-♤  6- -♤  8- -♤  J- -♤  4- -♤  K- -♤  9- -♤  K- -♤  3- -♤  8- -♤  7-
-♤  5- -♤  9- -♤  6- -♤  A- -♤  7- -♤  K- -♤  2- -♤  Q- -♤  8- -♤  5-
-♤  6- -♤  2- -♤  A- -♤  6- -♤  K- -♤  9- -♤  A- -♤  9- -♤  J- -♤ 10-
-♤  K- -♤  5- -♤  6- -♤  6-  ♤ 10   ♤  5   ♤ 10   ♤  J   ♤  5   ♤  9 
 ♤  Q   ♤  8   ♤  4   ♤  8                                     

Game#956 [Lost] Score: 464   Moves: 36   Completed: 0
-♤ 10- -♤  J- -♤  K- -♤  2- -♤  K- -♤  5- -♤  Q- -♤  3- -♤  2- -♤  9-
-♤ 10- -♤  3- -♤ 10- -♤  J- -♤  J- -♤  5- -♤  8- -♤  9- -♤  6- -♤  5-
-♤  A- -♤  J- -♤  2- -♤  5- -♤  Q- -♤  2- -♤  A- -♤ 10- -♤  Q- -♤ 10-
-♤  8- -♤  A- -♤  K- -♤  7- -♤  5- -♤  9- -♤  J- -♤  9- -♤  9- -♤  Q-
-♤  6- -♤  K- -♤  4- -♤  4-  ♤  3   ♤  7   ♤  K   ♤  7   ♤  Q   ♤  8 
 ♤  3   ♤  3   ♤  3   ♤  A                                           
Game#957 Score: 500   Moves: 0   Completed: 0   Undealt: 5
Game#957 [Lost] Score: 465   Moves: 35   Completed: 0
-♤  4- -♤  6- -♤  8- -♤  Q- -♤  5- -♤  A- -♤  9- -♤ 10- -♤  A- -♤  A-
-♤  6- -♤  4- -♤  9- -♤  5- -♤  2- -♤  7- -♤  Q- -♤  3- -♤  K- -♤  J-
-♤  A- -♤  4- -♤  8- -♤  2- -♤  2- -♤  2- -♤  Q- -♤  J- -♤  4- -♤  3-
-♤  7- -♤  Q- -♤  4- -♤  6- -♤ 10- -♤  K- -♤  K- -♤  K- -♤  8- -♤  K-
-♤  8- -♤ 10- -♤  K- -♤  2-  ♤  6   ♤  7   ♤  5   ♤  6   ♤  3   ♤  A 
 ♤  2   ♤  5   ♤  6   ♤  Q                                     

Game#972 [Lost] Score: 490   Moves: 10   Completed: 0
-♤  7- -♤  A- -♤  K- -♤  K- -♤ 10- -♤  Q- -♤  Q- -♤  6- -♤  K- -♤  7-
-♤  3- -♤  7- -♤  J- -♤  8- -♤  3- -♤  2- -♤  4- -♤  6- -♤  8- -♤  7-
-♤  8- -♤  J- -♤  4- -♤  9- -♤  A- -♤  7- -♤  A- -♤ 10- -♤  Q- -♤  9-
-♤  J- -♤  J- -♤  7- -♤  K- -♤  A- -♤  4- -♤  3- -♤ 10- -♤  8- -♤  2-
-♤  6- -♤ 10- -♤  K- -♤  J-  ♤  5   ♤  A   ♤  2   ♤  2   ♤  4   ♤  6 
 ♤  3   ♤  5   ♤  5   ♤  8                                           
Game#973 Score: 500   Moves: 0   Completed: 0   Undealt: 5
Game#973 [Lost] Score: 467   Moves: 33   Completed: 0
-♤  J- -♤  J- -♤  4- -♤  2- -♤  8- -♤  3- -♤  9- -♤  A- -♤  A- -♤  Q-
-♤  3- -♤  2- -♤  4- -♤  9- -♤  7- -♤  K- -♤ 10- -♤  5- -♤  4- -♤  J-
-♤  8- -♤  J- -♤  2- -♤  7- -♤  9- -♤  6- -♤  3- -♤ 10- -♤ 10- -♤  2-
-♤  9- -♤  2- -♤  9- -♤  7- -♤  Q- -♤  8- -♤  K- -♤  6- -♤  3- -♤  9-
-♤  8- -♤  3- -♤ 10- -♤  5-  ♤  4   ♤  Q   ♤  4   ♤  5   ♤  A   ♤  2 
 ♤  4   ♤  5   ♤  Q   ♤  5                                     

Game#990 [Lost] Score: 553   Moves: 48   Completed: 1
-♤  K- -♤  3- -♤ 10- -♤  6- -♤  3- -♤  Q- -♤  9- -♤  A- -♤  2- -♤  J-
-♤  9- -♤  4- -♤  K- -♤  6- -♤ 10- -♤  K- -♤  8- -♤  A- -♤  J- -♤  6-
-♤  8- -♤  4- -♤  Q- -♤ 10- -♤  2- -♤  A- -♤  6- -♤  2- -♤  A- -♤  K-
-♤  5- -♤  5- -♤  8- -♤ 10- -♤  4- -♤  J- -♤  2- -♤  A- -♤  3- -♤  A-
-♤  Q- -♤  J- -♤  2- -♤  Q-  ♤  5   ♤  9   ♤  7   ♤  K   ♤  8   ♤  A 
 ♤  3   ♤  K   ♤  5   ♤  8                                           
Game#991 Score: 500   Moves: 0   Completed: 0   Undealt: 5
Game#991 [Lost] Score: 465   Moves: 35   Completed: 0
-♤  4- -♤  6- -♤  J- -♤  4- -♤  Q- -♤  3- -♤  J- -♤  3- -♤  6- -♤  Q-
-♤  8- -♤  5- -♤  2- -♤ 10- -♤  7- -♤  8- -♤  Q- -♤  4- -♤  4- -♤ 10-
-♤  K- -♤  9- -♤  9- -♤ 10- -♤  6- -♤  5- -♤  A- -♤  3- -♤  7- -♤  A-
-♤  9- -♤  Q- -♤  2- -♤  Q- -♤  K- -♤  2- -♤  7- -♤  A- -♤  5- -♤  8-
-♤  A- -♤  6- -♤  K- -♤  5-  ♤  Q   ♤  9   ♤  3   ♤  7   ♤  8   ♤  J 
 ♤ 10   ♤  7   ♤  8   ♤  9                                     